In [358]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
appended_data = []
for year in range(1985, 2020):
    result = pd.DataFrame()
    url = 'https://www.basketball-reference.com/awards/awards_{}.html'.format(year)
    url2 = 'https://www.basketball-reference.com/leagues/NBA_{}_per_game.html'.format(year)
    url3 = 'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'.format(year)
    
    
    #**********************EXTRACT DATA FROM FIRST URL**********************
    temp = pd.read_html(url, header=[1])
    temp[0].dropna(axis=1,inplace=True)
    temp = temp[0][0:10]
    temp.insert(1, 'Year', year)
    temp.insert(2, 'MVP', [1,0,0,0,0,0,0,0,0,0])
    temp.columns = temp.columns.str.strip()
    temp = temp.round(4)
    #**********************EXTRACT DATA FROM SECOND URL**********************
    temp2 = pd.read_html(url2)
    temp2 = temp2[0].dropna(axis=1,how='all')
    temp2['Player'] = temp2['Player'].str.replace('*','')
    temp2.columns = temp2.columns.str.strip()
    temp2 = temp2[temp2.Pos != 'Pos'] #get rid of random header rows
    temp2 = temp2.round(4)
    #Find intersecting columns
    cols = list(np.intersect1d(temp.columns, temp2.columns))
    #Remove cols temporarily that don't involve numbers because casting
    #will add them back later when doing 'on = cols'
    cols.remove('Player')
    cols.remove('Tm')
    #need to convert cols to float
    temp[cols] = temp[cols].astype(float)
    temp2[cols] = temp2[cols].astype(float) 
    
    #add qualitative analysis back for on = within merge function
    cols.append('Player')
    cols.append('Tm')
    
    #MERGE TEMP AND TEMP2
    result = temp.merge(temp2, how='inner', on=cols)
    if (year == 1985):
        result.to_csv('csv/TEMP1_2.csv')
    #**********************EXTRACT DATA FROM THIRD URL**********************
    temp3 = pd.read_html(url3)
    temp3 = temp3[0].dropna(axis=1, how='all')
    temp3 = temp3[temp3.Pos != 'Pos']
    temp3['Player'] = temp3['Player'].str.replace('*','')
    temp3 = temp3.drop('Rk', 1)
    temp3.columns = temp3.columns.str.strip()
    temp3 = temp3.rename(columns={"MP": "TOT_MP"})
    temp3 = temp3.round(4)
    #Find intersecting columns
    cols = list(np.intersect1d(result.columns, temp3.columns))

    #Remove cols temporarily that don't involve numbers because casting
    #will add them back later when doing 'on = cols'
    cols.remove('Player')
    cols.remove('Tm')
    cols.remove('Pos')
    #need to convert cols to float
    temp3[cols] = temp3[cols].astype(float) 
    temp3.to_csv('csv/temp3.csv')
    #add qualitative analysis back for on = within merge function
    cols.append('Player')
    cols.append('Tm')
    cols.append('Pos')
    result = result.merge(temp3, how='inner', on=cols)
    #add to list of data
    appended_data.append(result)
df = pd.concat(appended_data)
#data frame complete

In [359]:
 saved_df = df #saves df from cell 1 into another df to prevent having to constantly run 

In [362]:
df = saved_df #move dataframe over 

df = saved_df

In [363]:
import requests
import re
from bs4 import BeautifulSoup


df.count()
df['PRE_ODDS'] = 0
#This only works for [1985,2019]. For 2020, we will have to do something else and make a current csv to grab 
#data for games played to most recent date

modify = df.reset_index(drop=True)
modify.at[245, 'Tm'] = 'DEN'

df = modify
for i in range(len(df.index)):
    team = df.iloc[i]['Tm']
    year = df.iloc[i]['Year']
    odds_url = "https://www.basketball-reference.com/teams/{}/{}.html".format(team,year)
    #print(odds_url)
    html = requests.get(odds_url).text
    soup = BeautifulSoup(html, "lxml")
    res = soup.find('a', {"href": "/leagues/NBA_{}_preseason_odds.html".format(year)} ).text
    #df['PRE_ODDS'][i] = re.findall(r'\d+', res)[0]
    df.at[i, 'PRE_ODDS'] = re.findall(r'\-?\d+', res)[0]
#display(df)
df.to_csv('csv/historical_mvp_data.csv')

In [348]:
df.to_csv('csv/historical_mvp_data.csv')